In [1]:
from confluent_kafka import Producer
import socket
#Initialize Your Parameters here - Keep the variable values as is for the ones you can't find on the Confluent-Kafka connection 
KAFKA_CONFIG = {
    "bootstrap.servers":"pkc-lzvrd.us-west4.gcp.confluent.cloud:9092", #Add server it gives you
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"KPCWQU76CLNVNZTT", #Add username given on site
    "sasl.password":"6jL00qPsdQ7PzmXuVhZm+jGtFyLu9vw26a1s8oEPJv93Dlvlk8dvkyKAT0v/1wR9", #Add password given on site
    "session.timeout.ms":"45000",
    "group.id":"python-group-1",    
    'auto.offset.reset': 'smallest',
    'client.id': socket.gethostname()
}

# Update your topic name
topic_name = "topic_0" #Add topic given on site
producer = Producer(KAFKA_CONFIG)
print("done")

done


%4|1696865578.496|CONFWARN|chads-mbp-4.wifi.local.cmu.edu#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
%4|1696865578.496|CONFWARN|chads-mbp-4.wifi.local.cmu.edu#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
%4|1696865578.496|CONFWARN|chads-mbp-4.wifi.local.cmu.edu#producer-1| [thrd:app]: Configuration property auto.offset.reset is a consumer property and will be ignored by this producer instance


In [3]:
import feedparser
import time
start_time = time.time()
# We are searching for Analytics in the news
feed_url = "https://news.google.com/rss/search?q=popular+cyber+attacks"
def extract_news_feed(feed_url):
    feed = feedparser.parse(feed_url)
    articles = []
    extracted_articles = set()
    start_time = time.time()
    while time.time() < (start_time + 60 * 2):  #Newsfeed for 5 Minutes
        for entry in feed.entries:
            link = entry.link
            title = entry.title.encode('ascii', 'ignore').decode()
            unique_id = f'{link}-{title}'
            if unique_id in extracted_articles:
                continue
            extracted_articles.add(unique_id)
            article_data = {"title": title, "link":link}
            if article_data is not None:
                producer.produce(topic_name, key=article_data["title"], value=article_data["link"])
        producer.flush()

    
extract_news_feed(feed_url)